In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("employee_retention_data.csv")

In [3]:
df.head()

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
0,13021.0,7,customer_service,28,89000.0,2014-03-24,2015-10-30
1,825355.0,7,marketing,20,183000.0,2013-04-29,2014-04-04
2,927315.0,4,marketing,14,101000.0,2014-10-13,NaN
3,662910.0,7,customer_service,20,115000.0,2012-05-14,2013-06-07
4,256971.0,2,data_science,23,276000.0,2011-10-17,2014-08-22


In [4]:
df.describe()

,employee_id,company_id,seniority,salary
count,24702.000000,24702.000000,24702.000000,24702.000000
mean,501604.403530,3.426969,14.127803,138183.345478
std,288909.026101,2.700011,8.089520,76058.184573
min,36.000000,1.000000,1.000000,17000.000000
25%,250133.750000,1.000000,7.000000,79000.000000
50%,500793.000000,2.000000,14.000000,123000.000000
75%,753137.250000,5.000000,21.000000,187000.000000
max,999969.000000,12.000000,99.000000,408000.000000


In [11]:
df["join_date"] = pd.to_datetime(df["join_date"])
df["quit_date"] = pd.to_datetime(df["quit_date"])

In [14]:
df.describe()

,employee_id,company_id,seniority,salary
count,24702.000000,24702.000000,24702.000000,24702.000000
mean,501604.403530,3.426969,14.127803,138183.345478
std,288909.026101,2.700011,8.089520,76058.184573
min,36.000000,1.000000,1.000000,17000.000000
25%,250133.750000,1.000000,7.000000,79000.000000
50%,500793.000000,2.000000,14.000000,123000.000000
75%,753137.250000,5.000000,21.000000,187000.000000
max,999969.000000,12.000000,99.000000,408000.000000


In [47]:
df.head()

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
0,13021.0,7,customer_service,28,89000.0,2014-03-24,2015-10-30
1,825355.0,7,marketing,20,183000.0,2013-04-29,2014-04-04
2,927315.0,4,marketing,14,101000.0,2014-10-13,NaT
3,662910.0,7,customer_service,20,115000.0,2012-05-14,2013-06-07
4,256971.0,2,data_science,23,276000.0,2011-10-17,2014-08-22


In [77]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


In [97]:
df2 = pd.DataFrame(columns=['company_id', 'employee_id', 'day'])
start_date = date(2011, 1, 24)
end_date = date(2015, 12, 14)
for single_date in daterange(start_date, end_date):
#     print(single_date)
    new_df = pd.DataFrame(df[df["join_date"] <= single_date].groupby("company_id").employee_id.count())
    new_df["day"] = single_date
    new_df = new_df.reset_index()
    df2 = pd.concat([df2, new_df])
    
    del_df = pd.DataFrame(df[df["quit_date"] <= single_date].groupby("company_id").quit_date.count())
    if len(del_df > 0):
        del_df["day"] = single_date
        del_df = del_df.reset_index()
        df2 = df2.merge(del_df, how = "left", on = (["company_id", "day"]))
        df2["quit_date"] = df2["quit_date"].replace(np.nan, 0)
        df2["employee_id"] = df2["employee_id"] - df2["quit_date"]
        df2 = df2[["company_id", "employee_id", "day"]]

In [99]:
df2.head()

,company_id,employee_id,day
0,1.0,25.0,2011-01-24
1,2.0,17.0,2011-01-24
2,3.0,9.0,2011-01-24
3,4.0,12.0,2011-01-24
4,5.0,5.0,2011-01-24


In [100]:
df.head()

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
0,13021.0,7,customer_service,28,89000.0,2014-03-24,2015-10-30
1,825355.0,7,marketing,20,183000.0,2013-04-29,2014-04-04
2,927315.0,4,marketing,14,101000.0,2014-10-13,NaT
3,662910.0,7,customer_service,20,115000.0,2012-05-14,2013-06-07
4,256971.0,2,data_science,23,276000.0,2011-10-17,2014-08-22


In [118]:
company_attrition = df.groupby("company_id").count().reset_index() 
company_attrition["Attrition"] = company_attrition.quit_date / company_attrition.join_date
company_attrition[["company_id", "Attrition", "join_date", "quit_date"]]

,company_id,Attrition,join_date,quit_date
0,1,0.544544,8486,4621
1,2,0.522501,4222,2206
2,3,0.556930,2749,1531
3,4,0.559166,2062,1153
4,5,0.560114,1755,983
5,6,0.551510,1291,712
6,7,0.565359,1224,692
7,8,0.553009,1047,579
8,9,0.550468,961,529
9,10,0.554913,865,480


In [120]:
dept_attrition = df.groupby("dept").count().reset_index() 
dept_attrition["Attrition"] = dept_attrition.quit_date / dept_attrition.join_date
dept_attrition[["dept", "Attrition", "join_date", "quit_date"]]

,dept,Attrition,join_date,quit_date
0,customer_service,0.554902,9180,5094
1,data_science,0.527273,3190,1682
2,design,0.563768,1380,778
3,engineer,0.512031,4613,2362
4,marketing,0.562993,3167,1783
5,sales,0.570933,3172,1811


In [122]:
company_dept_attrition = df.groupby(["company_id", "dept"]).count().reset_index() 
company_dept_attrition["Attrition"] = company_dept_attrition.quit_date / company_dept_attrition.join_date
company_dept_attrition[["company_id", "dept", "Attrition", "join_date", "quit_date"]]

,company_id,dept,Attrition,join_date,quit_date
0,1,customer_service,0.571112,3157,1803
1,1,data_science,0.523633,1079,565
2,1,design,0.539078,499,269
3,1,engineer,0.477041,1568,748
4,1,marketing,0.571429,1085,620
5,1,sales,0.561020,1098,616
6,2,customer_service,0.542636,1548,840
7,2,data_science,0.473592,568,269
8,2,design,0.565022,223,126
9,2,engineer,0.463209,829,384
